This Python Notebook generates features for "transactions_v2.csv" and exports a file into a folder called data "final_transactions". 
In this notebook, more changes to "transactions_v2" can be made. 
In the notebook "algorithm_solution" load "final_transactions" in.

In [1]:
#Import the relevant libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mpld3
import seaborn as sns
import matplotlib.dates as mdates
import time
from datetime import datetime
from pandas.lib import Timestamp


#Configure Panda
pd.options.display.width = 200

/Users/pablodepaz/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: The pandas.lib module is deprecated and will be removed in a future version. These are private functions and can be accessed from pandas._libs.lib instead
  # This is added back by InteractiveShellApp.init_path()


## 1. Data import and Feature Engineering

In [2]:
#Load transactions
transactions = pd.read_csv("data/transactions_v2.csv")

In [3]:
#Look at the first values in transactions:
print("Transactions:")
transactions.head()

Transactions:


,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
0,++6eU4LsQ3UQ20ILS7d99XK8WbiVgbyYL4FUgzZR134=,32,90,298,298,0,20170131,20170504,0
1,++lvGPJOinuin/8esghpnqdljm6NXS8m8Zwchc7gOeA=,41,30,149,149,1,20150809,20190412,0
2,+/GXNtXWQVfKrEDqYAzcSw2xSPYMKWNj22m+5XkVQZc=,36,30,180,180,1,20170303,20170422,0
3,+/w1UrZwyka4C9oNH3+Q8fUf3fD8R3EwWrx57ODIsqk=,36,30,180,180,1,20170329,20170331,1
4,+00PGzKTYqtnb65mPKPyeHXcZEwqiEzktpQksaaSC3c=,41,30,99,99,1,20170323,20170423,0


In [4]:
print('Number of rows & columns "TRANSACTIONS": ', transactions.shape)

Number of rows & columns "TRANSACTIONS":  (1431009, 9)


### 1.1. Feature 0. Feature computed as the difference between the "plan price" and "amount paid"

In [5]:
transactions['diff_plan_actual'] = transactions['plan_list_price'] - transactions['actual_amount_paid']
transactions.head()

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,diff_plan_actual
0,++6eU4LsQ3UQ20ILS7d99XK8WbiVgbyYL4FUgzZR134=,32,90,298,298,0,20170131,20170504,0,0
1,++lvGPJOinuin/8esghpnqdljm6NXS8m8Zwchc7gOeA=,41,30,149,149,1,20150809,20190412,0,0
2,+/GXNtXWQVfKrEDqYAzcSw2xSPYMKWNj22m+5XkVQZc=,36,30,180,180,1,20170303,20170422,0,0
3,+/w1UrZwyka4C9oNH3+Q8fUf3fD8R3EwWrx57ODIsqk=,36,30,180,180,1,20170329,20170331,1,0
4,+00PGzKTYqtnb65mPKPyeHXcZEwqiEzktpQksaaSC3c=,41,30,99,99,1,20170323,20170423,0,0


In [6]:
print('Number of rows & columns "TRANSACTIONS": ', transactions.shape)

Number of rows & columns "TRANSACTIONS":  (1431009, 10)


### 1.2. Feature 1: discount
I create a discount column to see how much discount was offered to the customer.

In [7]:
transactions['discount'] = transactions['plan_list_price'] - transactions['actual_amount_paid']

#transactions['discount'].unique() #Find the unique elements of an array/dataset

In [8]:
print('Number of rows & columns "TRANSACTIONS": ', transactions.shape)

Number of rows & columns "TRANSACTIONS":  (1431009, 11)


## 1.3. Feature 2 : is_discount
Feature to check whether the customer has availed any discount or not

In [9]:
transactions['is_discount'] = transactions.discount.apply(lambda x: 1 if x > 0 else 0)
#print(transactions['is_discount'].unique())

In [10]:
print('Number of rows & columns "TRANSACTIONS": ', transactions.shape)

Number of rows & columns "TRANSACTIONS":  (1431009, 12)


### 1.4. Feature 3: membership duration
Difference between transaction_date and membership_expire_date. We find the difference in terms of days and months. We keep this result as integer.

In [11]:
#First of all, transform the dates to "datetpye"
transactions['transaction_date'] = transactions.transaction_date.apply(lambda x: datetime.strptime(str(int(x)), "%Y%m%d").date() if pd.notnull(x) else "NAN" )
transactions['membership_expire_date'] = transactions.membership_expire_date.apply(lambda x: datetime.strptime(str(int(x)), "%Y%m%d").date() if pd.notnull(x) else "NAN" )
transactions.head()

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,diff_plan_actual,discount,is_discount
0,++6eU4LsQ3UQ20ILS7d99XK8WbiVgbyYL4FUgzZR134=,32,90,298,298,0,2017-01-31,2017-05-04,0,0,0,0
1,++lvGPJOinuin/8esghpnqdljm6NXS8m8Zwchc7gOeA=,41,30,149,149,1,2015-08-09,2019-04-12,0,0,0,0
2,+/GXNtXWQVfKrEDqYAzcSw2xSPYMKWNj22m+5XkVQZc=,36,30,180,180,1,2017-03-03,2017-04-22,0,0,0,0
3,+/w1UrZwyka4C9oNH3+Q8fUf3fD8R3EwWrx57ODIsqk=,36,30,180,180,1,2017-03-29,2017-03-31,1,0,0,0
4,+00PGzKTYqtnb65mPKPyeHXcZEwqiEzktpQksaaSC3c=,41,30,99,99,1,2017-03-23,2017-04-23,0,0,0,0


In [13]:
#--- difference in days ---
transactions['mem_duration'] = transactions.membership_expire_date - transactions.transaction_date
transactions['mem_duration'] = transactions['mem_duration'] / np.timedelta64(1, 'D')
transactions['mem_duration'] = transactions['mem_duration'].astype(int)

In [14]:
transactions.head()
#I am going to delete the rows whose mem_duration is bigger than 90 days (for example)
#UNDONE

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,diff_plan_actual,discount,is_discount,mem_duration
0,++6eU4LsQ3UQ20ILS7d99XK8WbiVgbyYL4FUgzZR134=,32,90,298,298,0,2017-01-31,2017-05-04,0,0,0,0,93
1,++lvGPJOinuin/8esghpnqdljm6NXS8m8Zwchc7gOeA=,41,30,149,149,1,2015-08-09,2019-04-12,0,0,0,0,1342
2,+/GXNtXWQVfKrEDqYAzcSw2xSPYMKWNj22m+5XkVQZc=,36,30,180,180,1,2017-03-03,2017-04-22,0,0,0,0,50
3,+/w1UrZwyka4C9oNH3+Q8fUf3fD8R3EwWrx57ODIsqk=,36,30,180,180,1,2017-03-29,2017-03-31,1,0,0,0,2
4,+00PGzKTYqtnb65mPKPyeHXcZEwqiEzktpQksaaSC3c=,41,30,99,99,1,2017-03-23,2017-04-23,0,0,0,0,31


### 1.5. Feature Engineering: "trans1". Hot-encode "payment_method_id" feature and test algorithm

In [16]:
#One-hot encode the payment_method_id. 
#Instead of having a variable called payment_method_id with values from 2-41, the alorithm performs better with 0's and 1's -> onehot encoding

trans1 = transactions

#One-hot encode payment_method_id and save it into payment_method_id_encode
payment_method_id_encode = pd.get_dummies(trans1['payment_method_id'], prefix='payment_method_id')

#Drop variable payment_method_id in trans1, as it is no longer needed
trans1 = trans1.drop('payment_method_id', axis=1)

#Join the encoded payment_method_id_encode
trans1 = trans1.join(payment_method_id_encode)

trans1.head()

,msno,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,diff_plan_actual,discount,...,payment_method_id_32,payment_method_id_33,payment_method_id_34,payment_method_id_35,payment_method_id_36,payment_method_id_37,payment_method_id_38,payment_method_id_39,payment_method_id_40,payment_method_id_41
0,++6eU4LsQ3UQ20ILS7d99XK8WbiVgbyYL4FUgzZR134=,90,298,298,0,2017-01-31,2017-05-04,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,++lvGPJOinuin/8esghpnqdljm6NXS8m8Zwchc7gOeA=,30,149,149,1,2015-08-09,2019-04-12,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,+/GXNtXWQVfKrEDqYAzcSw2xSPYMKWNj22m+5XkVQZc=,30,180,180,1,2017-03-03,2017-04-22,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,+/w1UrZwyka4C9oNH3+Q8fUf3fD8R3EwWrx57ODIsqk=,30,180,180,1,2017-03-29,2017-03-31,1,0,0,...,0,0,0,0,1,0,0,0,0,0
4,+00PGzKTYqtnb65mPKPyeHXcZEwqiEzktpQksaaSC3c=,30,99,99,1,2017-03-23,2017-04-23,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [17]:
#I get rid of columns whose value is below 30 as not significative
trans1 = trans1.drop("payment_method_id_2", axis=1)
trans1 = trans1.drop("payment_method_id_3", axis=1)
trans1 = trans1.drop("payment_method_id_5", axis=1)
trans1 = trans1.drop("payment_method_id_6", axis=1)
trans1 = trans1.drop("payment_method_id_8", axis=1)
trans1 = trans1.drop("payment_method_id_10", axis=1)
trans1 = trans1.drop("payment_method_id_11", axis=1)
trans1 = trans1.drop("payment_method_id_12", axis=1)
trans1 = trans1.drop("payment_method_id_13", axis=1)
trans1 = trans1.drop("payment_method_id_14", axis=1)
trans1 = trans1.drop("payment_method_id_15", axis=1)
trans1 = trans1.drop("payment_method_id_16", axis=1)
trans1 = trans1.drop("payment_method_id_17", axis=1)
trans1 = trans1.drop("payment_method_id_18", axis=1)
trans1 = trans1.drop("payment_method_id_19", axis=1)
trans1 = trans1.drop("payment_method_id_20", axis=1)
trans1 = trans1.drop("payment_method_id_21", axis=1)
trans1 = trans1.drop("payment_method_id_22", axis=1)
trans1 = trans1.drop("payment_method_id_23", axis=1)
trans1 = trans1.drop("payment_method_id_24", axis=1)
trans1 = trans1.drop("payment_method_id_25", axis=1)
trans1 = trans1.drop("payment_method_id_26", axis=1)
trans1 = trans1.drop("payment_method_id_27", axis=1)
trans1 = trans1.drop("payment_method_id_28", axis=1)
trans1 = trans1.drop("payment_method_id_29", axis=1)

In [18]:
trans1.head()

,msno,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,diff_plan_actual,discount,...,payment_method_id_32,payment_method_id_33,payment_method_id_34,payment_method_id_35,payment_method_id_36,payment_method_id_37,payment_method_id_38,payment_method_id_39,payment_method_id_40,payment_method_id_41
0,++6eU4LsQ3UQ20ILS7d99XK8WbiVgbyYL4FUgzZR134=,90,298,298,0,2017-01-31,2017-05-04,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,++lvGPJOinuin/8esghpnqdljm6NXS8m8Zwchc7gOeA=,30,149,149,1,2015-08-09,2019-04-12,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,+/GXNtXWQVfKrEDqYAzcSw2xSPYMKWNj22m+5XkVQZc=,30,180,180,1,2017-03-03,2017-04-22,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,+/w1UrZwyka4C9oNH3+Q8fUf3fD8R3EwWrx57ODIsqk=,30,180,180,1,2017-03-29,2017-03-31,1,0,0,...,0,0,0,0,1,0,0,0,0,0
4,+00PGzKTYqtnb65mPKPyeHXcZEwqiEzktpQksaaSC3c=,30,99,99,1,2017-03-23,2017-04-23,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [19]:
print('Number of rows & columns "TRANSACTIONS": ', transactions.shape)
print('Number of rows & columns "TRANSACTIONS": ', trans1.shape)
#Same number of rows!

Number of rows & columns "TRANSACTIONS":  (1431009, 13)
Number of rows & columns "TRANSACTIONS":  (1431009, 24)


### 1.6. Feature Engineering: "trans2". Hot-encode "payment_plan_days" feature and test algorithm

In [23]:
trans2 = trans1

payment_plan_days_encode = pd.get_dummies(trans2['payment_plan_days'], prefix='payment_plan_days')

#Drop variable payment_method_id in trans1, as it is no longer needed
trans2 = trans2.drop('payment_plan_days', axis=1)

#Join the encoded payment_method_id_encode
trans2 = trans2.join(payment_plan_days_encode)

trans2.head()

,msno,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,diff_plan_actual,discount,is_discount,...,payment_plan_days_230,payment_plan_days_240,payment_plan_days_270,payment_plan_days_360,payment_plan_days_365,payment_plan_days_395,payment_plan_days_400,payment_plan_days_410,payment_plan_days_415,payment_plan_days_450
0,++6eU4LsQ3UQ20ILS7d99XK8WbiVgbyYL4FUgzZR134=,298,298,0,2017-01-31,2017-05-04,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,++lvGPJOinuin/8esghpnqdljm6NXS8m8Zwchc7gOeA=,149,149,1,2015-08-09,2019-04-12,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,+/GXNtXWQVfKrEDqYAzcSw2xSPYMKWNj22m+5XkVQZc=,180,180,1,2017-03-03,2017-04-22,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,+/w1UrZwyka4C9oNH3+Q8fUf3fD8R3EwWrx57ODIsqk=,180,180,1,2017-03-29,2017-03-31,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,+00PGzKTYqtnb65mPKPyeHXcZEwqiEzktpQksaaSC3c=,99,99,1,2017-03-23,2017-04-23,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
#I get rid of columns whose value is below 30 as not significative
trans2 = trans2.drop("payment_plan_days_1", axis=1)
trans2 = trans2.drop("payment_plan_days_3", axis=1)
trans2 = trans2.drop("payment_plan_days_10", axis=1)
trans2 = trans2.drop("payment_plan_days_14", axis=1)
trans2 = trans2.drop("payment_plan_days_31", axis=1)
trans2 = trans2.drop("payment_plan_days_35", axis=1)
trans2 = trans2.drop("payment_plan_days_45", axis=1)
trans2 = trans2.drop("payment_plan_days_60", axis=1)
trans2 = trans2.drop("payment_plan_days_70", axis=1)
trans2 = trans2.drop("payment_plan_days_80", axis=1)
trans2 = trans2.drop("payment_plan_days_100", axis=1)
trans2 = trans2.drop("payment_plan_days_110", axis=1)
trans2 = trans2.drop("payment_plan_days_120", axis=1)
trans2 = trans2.drop("payment_plan_days_200", axis=1)
trans2 = trans2.drop("payment_plan_days_230", axis=1)
trans2 = trans2.drop("payment_plan_days_240", axis=1)
trans2 = trans2.drop("payment_plan_days_270", axis=1)
trans2 = trans2.drop("payment_plan_days_360", axis=1)
trans2 = trans2.drop("payment_plan_days_365", axis=1)
trans2 = trans2.drop("payment_plan_days_400", axis=1)
trans2 = trans2.drop("payment_plan_days_415", axis=1)
trans2 = trans2.drop("payment_plan_days_450", axis=1)

In [25]:
trans2.head()

,msno,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,diff_plan_actual,discount,is_discount,...,payment_method_id_41,payment_plan_days_0,payment_plan_days_7,payment_plan_days_21,payment_plan_days_30,payment_plan_days_90,payment_plan_days_180,payment_plan_days_195,payment_plan_days_395,payment_plan_days_410
0,++6eU4LsQ3UQ20ILS7d99XK8WbiVgbyYL4FUgzZR134=,298,298,0,2017-01-31,2017-05-04,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,++lvGPJOinuin/8esghpnqdljm6NXS8m8Zwchc7gOeA=,149,149,1,2015-08-09,2019-04-12,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
2,+/GXNtXWQVfKrEDqYAzcSw2xSPYMKWNj22m+5XkVQZc=,180,180,1,2017-03-03,2017-04-22,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,+/w1UrZwyka4C9oNH3+Q8fUf3fD8R3EwWrx57ODIsqk=,180,180,1,2017-03-29,2017-03-31,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,+00PGzKTYqtnb65mPKPyeHXcZEwqiEzktpQksaaSC3c=,99,99,1,2017-03-23,2017-04-23,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0


In [26]:
print('Number of rows & columns "TRANSACTIONS": ', transactions.shape)
print('Number of rows & columns "TRANSACTIONS": ', trans2.shape)
#Same number of rows!

Number of rows & columns "TRANSACTIONS":  (1431009, 13)
Number of rows & columns "TRANSACTIONS":  (1431009, 32)


### 1.7. Feature Engineering: "trans3". Hot-encode "plan_list_price" feature and test algorithm

In [27]:
trans3 = trans2

plan_list_price_encode = pd.get_dummies(trans3['plan_list_price'], prefix='plan_list_price')

#Drop variable payment_method_id in trans1, as it is no longer needed
trans3 = trans3.drop('plan_list_price', axis=1)

#Join the encoded payment_method_id_encode
trans3 = trans3.join(plan_list_price_encode)

trans3.head()

,msno,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,diff_plan_actual,discount,is_discount,mem_duration,...,plan_list_price_1000,plan_list_price_1150,plan_list_price_1200,plan_list_price_1260,plan_list_price_1299,plan_list_price_1300,plan_list_price_1399,plan_list_price_1599,plan_list_price_1788,plan_list_price_2000
0,++6eU4LsQ3UQ20ILS7d99XK8WbiVgbyYL4FUgzZR134=,298,0,2017-01-31,2017-05-04,0,0,0,0,93,...,0,0,0,0,0,0,0,0,0,0
1,++lvGPJOinuin/8esghpnqdljm6NXS8m8Zwchc7gOeA=,149,1,2015-08-09,2019-04-12,0,0,0,0,1342,...,0,0,0,0,0,0,0,0,0,0
2,+/GXNtXWQVfKrEDqYAzcSw2xSPYMKWNj22m+5XkVQZc=,180,1,2017-03-03,2017-04-22,0,0,0,0,50,...,0,0,0,0,0,0,0,0,0,0
3,+/w1UrZwyka4C9oNH3+Q8fUf3fD8R3EwWrx57ODIsqk=,180,1,2017-03-29,2017-03-31,1,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
4,+00PGzKTYqtnb65mPKPyeHXcZEwqiEzktpQksaaSC3c=,99,1,2017-03-23,2017-04-23,0,0,0,0,31,...,0,0,0,0,0,0,0,0,0,0


In [17]:
print('Number of rows & columns "TRANSACTIONS": ', transactions.shape)
print('Number of rows & columns "TRANSACTIONS": ', trans3.shape)
#Same number of rows!

Number of rows & columns "TRANSACTIONS":  (1431009, 12)
Number of rows & columns "TRANSACTIONS":  (1431009, 125)


### 1.8. Feature 4. autorenew_&_not_cancel
A binary feature to see whether mebers have auto renewed and not cancelled at the same time:
- auto_renew = 1 and
- is_cancel = 0

In [31]:
trans2['autorenew_and_not_cancel'] = ((trans2.is_auto_renew == 1) == (trans2.is_cancel == 0)).astype(np.int8)

In [33]:
#trans2.head()

### 1.9. Feature 5: notAutorenew_&_cancel
Binary feature to predict possible churning if
- auto_renew = 0 and
- is_cancel = 1

In [34]:
trans2['notAutorenew_and_cancel'] = ((trans2.is_auto_renew == 0) == (trans2.is_cancel == 1)).astype(np.int8)

In [35]:
trans2.head()

,msno,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,diff_plan_actual,discount,is_discount,...,payment_plan_days_7,payment_plan_days_21,payment_plan_days_30,payment_plan_days_90,payment_plan_days_180,payment_plan_days_195,payment_plan_days_395,payment_plan_days_410,autorenew_and_not_cancel,notAutorenew_and_cancel
0,++6eU4LsQ3UQ20ILS7d99XK8WbiVgbyYL4FUgzZR134=,298,298,0,2017-01-31,2017-05-04,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,++lvGPJOinuin/8esghpnqdljm6NXS8m8Zwchc7gOeA=,149,149,1,2015-08-09,2019-04-12,0,0,0,0,...,0,0,1,0,0,0,0,0,1,1
2,+/GXNtXWQVfKrEDqYAzcSw2xSPYMKWNj22m+5XkVQZc=,180,180,1,2017-03-03,2017-04-22,0,0,0,0,...,0,0,1,0,0,0,0,0,1,1
3,+/w1UrZwyka4C9oNH3+Q8fUf3fD8R3EwWrx57ODIsqk=,180,180,1,2017-03-29,2017-03-31,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,+00PGzKTYqtnb65mPKPyeHXcZEwqiEzktpQksaaSC3c=,99,99,1,2017-03-23,2017-04-23,0,0,0,0,...,0,0,1,0,0,0,0,0,1,1


### 1.10. Check dataset

In [37]:
#Check columns
print(len(trans2.columns))
trans2.columns

34


Index(['msno', 'plan_list_price', 'actual_amount_paid', 'is_auto_renew', 'transaction_date', 'membership_expire_date', 'is_cancel', 'diff_plan_actual', 'discount', 'is_discount', 'mem_duration',
       'payment_method_id_30', 'payment_method_id_31', 'payment_method_id_32', 'payment_method_id_33', 'payment_method_id_34', 'payment_method_id_35', 'payment_method_id_36', 'payment_method_id_37',
       'payment_method_id_38', 'payment_method_id_39', 'payment_method_id_40', 'payment_method_id_41', 'payment_plan_days_0', 'payment_plan_days_7', 'payment_plan_days_21', 'payment_plan_days_30',
       'payment_plan_days_90', 'payment_plan_days_180', 'payment_plan_days_195', 'payment_plan_days_395', 'payment_plan_days_410', 'autorenew_and_not_cancel', 'notAutorenew_and_cancel'],
      dtype='object')

#### 1.7.1. Check if missing values

In [38]:
trans2.isnull().sum()
#We should get 0!

msno                        0
plan_list_price             0
actual_amount_paid          0
is_auto_renew               0
transaction_date            0
membership_expire_date      0
is_cancel                   0
diff_plan_actual            0
discount                    0
is_discount                 0
mem_duration                0
payment_method_id_30        0
payment_method_id_31        0
payment_method_id_32        0
payment_method_id_33        0
payment_method_id_34        0
payment_method_id_35        0
payment_method_id_36        0
payment_method_id_37        0
payment_method_id_38        0
payment_method_id_39        0
payment_method_id_40        0
payment_method_id_41        0
payment_plan_days_0         0
payment_plan_days_7         0
payment_plan_days_21        0
payment_plan_days_30        0
payment_plan_days_90        0
payment_plan_days_180       0
payment_plan_days_195       0
payment_plan_days_395       0
payment_plan_days_410       0
autorenew_and_not_cancel    0
notAutoren

## 2. Export file
Recall, it can take some minutes!

In [39]:
trans3.to_csv('data/final_transactions.csv', index=False)
print('Done!')

Done!
